# Overview of the Lower-Level nmrsim API

This notebook gives a tour of some of the lower-level API functions. We recommend that you start with the [**API Introduction**](./1-Introduction-to-API.ipynb) notebook for a higher-level overview.

In [ ]:
import os
import sys
import numpy as np
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 300
%matplotlib inline

In [ ]:
%config InlineBackend.figure_format = 'svg'  # makes inline plot look less blurry

In [ ]:
home_path = os.path.abspath(os.path.join('..', '..', '..'))
if home_path not in sys.path:
    sys.path.append(home_path)

In [ ]:
tests_path = os.path.abspath(os.path.join('..', '..', '..', 'tests'))
if tests_path not in sys.path:
    sys.path.append(tests_path)

In [ ]:
from nmrsim import plt, qm

## Scenario: user wants to plot a spectrum for an ABX 3-spin system.


The API-Introduction notebook shows a simulation of an ABX 3-spin system using the SpinSystem class. Here, the simulation will be performed first with higher-level functions that take frequency(v) and intensity(J) arguments and return peaklists.

In [ ]:
# This dataset is for the vinyl group of vinyl acetate, as used in:
# http://www.users.csbsju.edu/~frioux/nmr/ABC-NMR-Tensor.pdf
def rioux():
    v = np.array([430.0, 265.0, 300.0])
    J = np.zeros((3, 3))
    J[0, 1] = 7.0
    J[0, 2] = 15.0
    J[1, 2] = 1.50
    J = J + J.T
    return v, J

In [ ]:
v, J = rioux()
print('v: ', v)  # frequencies in Hz
print('J: \n', J)  # matrix of coupling constants

The J matrix is constructed so that J[a, b] is the coupling constant between v[a] and v[b]. The diagonal elements should be 0.

### Method 1: using qm_spinsystem

In [ ]:
abx_system = qm.qm_spinsystem(v, J)
abx_system

In [ ]:
plt.mplplot(abx_system, y_max=0.2);

*{`qm_spinsystem` is a wrapper that selects one of two functions to perform the calculation: `qm.secondorder_dense` and `qm.secondorder_sparse`. With the default qm_spinsystem keyword arguments `cache=True` and `sparse=True`, the faster function `secondorder_sparse` is used. However, if at some point the sparse library becomes unavailable, or if caching of partial solutions is not possible, the slower `secondorder_dense` function will be used. These functions can also be used as direct swap-ins for `qm_spinsystem`.}*

### Method 2: via the spin Hamiltonian
This is not recommended for casual users, but may be of interest for teaching NMR theory, or if you want to take control of the process (e.g. obtain a Hamiltonian, and then simulate a spin pulse with it {a feature not currently implemented in `nmrsim`}). A description of the math behind the qm simulations is in the **qm_explanation.ipynb notebook** (currently under construction).

There are two versions of the Hamiltonian constructor. `qm.hamiltonian_sparse` uses cached sparse arrays for faster speed, and `qm.hamiltonian_dense` does not. Here we will use the former.

In [ ]:
H = qm.hamiltonian_sparse(v, J)
print(H)
print(H.todense())

SpinSystem defaults to second-order simulation of a spin system. If the SpinSystem object is instantiated with the `second_order=False` keyword argument, or if the SpinSystem.second_order attribute is set to `False`, first-order simulation will be performed instead.

`qm.solve_hamilton` accepts a *dense* Hamiltonian array and the number of spins in the system, to give a peaklist:

In [ ]:
peaklist = qm.solve_hamiltonian(H.todense(), nspins=3)
peaklist

To normalize the intensities so that they add up to 3 (the number of nuclei in the spin system), use `nmrsim.math.normalize_peaklist`:

In [ ]:
from nmrsim.math import normalize_peaklist
plist_normalized = normalize_peaklist(peaklist, 3)
plist_normalized

In [ ]:
plt.mplplot(plist_normalized, y_max=0.2);

### Method 3: using a discrete mathematical solution
The `nmrsim.discrete` module has discrete solutions for some common spin systems. Some are exact (such as discrete.AB for AB quartets) while others are approximations (e.g. `partial.ABX` for an ABX system) or return only part of the solution (e.g. `partial.AAXX` for an AA'XX' system).

The `partial.ABX` function uses an approximation that assumes the X nucleus is very far away in chemical shift from A and B. If accuracy is required, use a second-order calculation instead.

The functions in `nmrsim.discrete` also take different arguments than those usual throughout the rest of the nmrsim library. They are derived from similar functions in [Hans Reich's WINDNMR program](https://www.chem.wisc.edu/areas/reich/plt/windnmr.htm) and use similar inputs.

In [ ]:
from nmrsim.discrete import ABX
help(ABX)

In [ ]:
peaklist = ABX(1.5, 7, 15, (265-300), ((265+300)/2), 430)  # JAB, JAX, JBX, Vab, Vcentr, vx
plt.mplplot(peaklist, y_max=0.2);

### Method 4: a first-order simulation 
The same v/J arguments can be used by `nmrsim.firstorder.first_order_spin_system` to return a peaklist for a first-order simulation:

In [ ]:
from nmrsim.firstorder import first_order_spin_system
peaklist = first_order_spin_system(v, J)
plt.mplplot(peaklist, y_max = 0.2);

Individual multiplets can also be modeled using `nmrsim.firstorder.multiplet`. For example, for the X part of the ABX system as a first-order signal, i.e. 430 Hz, 1H, dd, *J* = 15, 7 Hz:

In [ ]:
from nmrsim.firstorder import multiplet
X = multiplet((430, 1), [(15, 1), (7, 1)])  # args (frequency, integration), [(J, # of couplings)...]
print(X)
plt.mplplot(X, y_max=0.2);

## Scenario: modeling DNMR spectra

The nmrsim.dnmr module provides functions as well as classes for the computation of DNMR lineshapes. Currently there are models for two systems: two uncoupled spins (`dnmr.dnmr_two_singlets`), and two coupled spins (`dnmr.dnmr_AB`, i.e an AB or AX system at the slow-exchange limit). 

In [ ]:
from nmrsim.dnmr import dnmr_two_singlets
help(dnmr_two_singlets)

In [ ]:
frequency, intensity = dnmr_two_singlets(165, 135, 1.5, 0.5, 0.5, 0.5) # va, vb, ka, wa, wb, pa
frequency[:10], intensity[:10]

To plot lineshape data such as the above (a pair of lists, one for all x coordinates and one for the corresponding y coordinates), you can use the visualization library of your choice. For a quick matplotlib representation, you can use `nmrsim.plt.mplplot_lineshape`:

In [ ]:
from nmrsim.plt import mplplot_lineshape
mplplot_lineshape(frequency, intensity);

Coalescence for this system occurs at k ~= 65.9 s<sup>-1</sup>:

In [ ]:
mplplot_lineshape(*dnmr_two_singlets(165, 135, 65.9, 0.5, 0.5, 0.5));